# Problem Statement : Hospital Patient Data Analysis

In [ ]:
import pandas as pd

# 1.Loading the datasets and showing summary with info().

patient_df = pd.read_csv("Patient_Data.csv")
billing_df = pd.read_csv("Billing_Data.csv")

In [ ]:
# Display structure and data types
patient_df.info()

In [ ]:
billing_df.info()

In [ ]:
# 2.Selecting only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
billing_columns = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_df = patient_df[billing_columns]
patient_df


In [ ]:
# 3.Droping administrative columns like ['ReceptionistID', 'CheckInTime'].
admin_cols = ['ReceptionistID', 'CheckInTime']
patient_df.drop(columns=admin_cols, inplace=True, errors='ignore' )
print(patient_df.head())

In [ ]:
# 4.Finding total bill amount per department by using groupby().
department_revenue = patient_df.groupby('Department')['BillAmount'].sum()

print(department_revenue)


In [ ]:
# 5.Removing duplicate patient records based on PatientID.
patient_df.drop_duplicates(subset='PatientID', inplace=True)

billing_df.drop_duplicates(subset='PatientID',inplace=True )


In [ ]:
# 6.Filling missing BillAmount values with the mean bill amount.
mean_bill = round(patient_df['BillAmount'].mean(), 1)
bill_col_index = patient_df.columns.get_loc('BillAmount')

for i in range(len(patient_df)):
    if pd.isna(patient_df.iloc[i, bill_col_index]):
        patient_df.iloc[i, bill_col_index] = mean_bill
    patient_df.iloc[i, bill_col_index] = round(patient_df.iloc[i, bill_col_index], 1)


In [ ]:
# 7.Merging the both billing dataset + patient dataset based on PatientID.

hospital_df = pd.merge(
    patient_df,
    billing_df,
    on='PatientID',
    how='left'
)
hospital_df.head()


In [ ]:
# 8.Concatenating an additional DataFrame that contains new patients for the current week (row-wise).
# Example new weekly patients DataFrame
new_patients_df = pd.DataFrame({
    'PatientID': [106,107],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. Rao', 'Dr. Mehta'],
    'BillAmount': [4500, 5200] 
})

hospital_df = pd.concat([hospital_df, new_patients_df], axis=0, ignore_index=True)
hospital_df

In [ ]:
# 9.Concatenating new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
new_billing = pd.DataFrame({
    'PatientID'        : [106, 107],
    'InsuranceCovered' : [1800, 2100],
    'FinalAmount'      : [2700, 3100]
})

hospital_df = hospital_df.set_index('PatientID')
hospital_df.update(new_billing.set_index('PatientID'))   # column‑wise fill
hospital_df.reset_index(inplace=True)

In [ ]:
# 10.Final Cleaned Dataset
hospital_df

In [ ]:
# Total revenue (BillAmount) per department
dept_rev = hospital_df.groupby('Department')['BillAmount'].sum()
print("\nRevenue per department (BillAmount):")
print(dept_rev)

# Doctor‑wise average final amount received from patients
doc_perf = hospital_df.groupby('Doctor')['FinalAmount'].mean()
print("\nDoctor performance – average FinalAmount:")
print(doc_perf)
